<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# Convert LaTeX Sentence to SymPy Expression

## Author: Ken Sible

## The following module will demonstrate a recursive descent parser for LaTeX.

### NRPy+ Source Code for this module:
1. [latex_parser.py](../edit/latex_parser.py); [\[**tutorial**\]](Tutorial-LaTeX_SymPy_Conversion.ipynb) The latex_parser.py script will convert a LaTeX sentence to a SymPy expression using the following function: parse(sentence).

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

1. [Step 1](#intro): Introduction: Lexical Analysis and Syntax Analysis
1. [Step 2](#sandbox): Demonstration and Sandbox (LaTeX Parser)
1. [Step 3](#tensor): Tensor Support with Einstein Notation (WIP)
1. [Step 4](#latex_pdf_output): $\LaTeX$ PDF Output

<a id='intro'></a>

# Step 1: Lexical Analysis and Syntax Analysis \[Back to [top](#toc)\]
$$\label{intro}$$

In the following section, we discuss [lexical analysis](https://en.wikipedia.org/wiki/Lexical_analysis) (lexing) and [syntax analysis](https://en.wikipedia.org/wiki/Parsing) (parsing). In the process of lexical analysis, a lexer will tokenize a character string, called a sentence, using substring pattern matching (or tokenizing). We implemented a regex-based lexer for NRPy+, which does pattern matching using a [regular expression](https://en.wikipedia.org/wiki/Regular_expression) for each token pattern. In the process of syntax analysis, a parser will receive a token iterator from the lexer and build a parse tree containing all syntactic information of the language, as specified by a [formal grammar](https://en.wikipedia.org/wiki/Formal_grammar). We implemented a [recursive descent parser](https://en.wikipedia.org/wiki/Recursive_descent_parser) for NRPy+, which will build a parse tree in [preorder](https://en.wikipedia.org/wiki/Tree_traversal#Pre-order_(NLR)), starting from the root [nonterminal](https://en.wikipedia.org/wiki/Terminal_and_nonterminal_symbols), using a [right recursive](https://en.wikipedia.org/wiki/Left_recursion) grammar. The following right recursive, [context-free grammar](https://en.wikipedia.org/wiki/Context-free_grammar) was written for parsing [LaTeX](https://en.wikipedia.org/wiki/LaTeX), adhering to the canonical (extended) [BNF](https://en.wikipedia.org/wiki/Backus%E2%80%93Naur_form) notation used for describing a context-free grammar:
```
<ROOT>       -> <EXPRESSION> | <STRUCTURE> { <LINE_BREAK> <STRUCTURE> }*
<STRUCTURE>  -> <CONFIG> | <ENVIROMENT> | <ASSIGNMENT>
<ENVIROMENT> -> <BEGIN_ALIGN> <ASSIGNMENT> { <LINE_BREAK> <ASSIGNMENT> }* <END_ALIGN>
<ASSIGNMENT> -> <VARIABLE> = <EXPRESSION>
<EXPRESSION> -> <TERM> { ( '+' | '-' ) <TERM> }*
<TERM>       -> <FACTOR> { [ '/' ] <FACTOR> }*
<FACTOR>     -> <BASE> { '^' <EXPONENT> }*
<BASE>       -> [ '-' ] ( <ATOM> | '(' <EXPRESSION> ')' | '[' <EXPRESSION> ']' )
<EXPONENT>   -> <BASE> | '{' <BASE> '}'
<ATOM>       -> <VARIABLE> | <NUMBER> | <COMMAND>
<VARIABLE>   -> <ARRAY> | <SYMBOL> [ '_' ( <SYMBOL> | <INTEGER> ) ]
<NUMBER>     -> <RATIONAL> | <DECIMAL> | <INTEGER>
<COMMAND>    -> <SQRT> | <FRAC>
<SQRT>       -> '\\sqrt' [ '[' <INTEGER> ']' ] '{' <EXPRESSION> '}'
<FRAC>       -> '\\frac' '{' <EXPRESSION> '}' '{' <EXPRESSION> '}'
<CONFIG>     -> '%' <ARRAY> '[' <INTEGER> ']' [ ':' <SYMMETRY> ] { ',' <ARRAY> '[' <INTEGER> ']' [ ':' <SYMMETRY> ] }*
<ARRAY>      -> ( <SYMBOL | <TENSOR> ) 
                    [ '_' ( <SYMBOL> | '{' { <SYMBOL> }+ '}' ) [ '^' ( <SYMBOL> | '{' { <SYMBOL> }+ '}' ) ]
                    | '^' ( <SYMBOL> | '{' { <SYMBOL> }+ '}' ) [ '_' ( <SYMBOL> | '{' { <SYMBOL> }+ '}' ) ] ]
```

<small>**Source**: Robert W. Sebesta. Concepts of Programming Languages. Pearson Education Limited, 2016.</small>

In [1]:
from latex_parser import * # Import NRPy+ module for lexing and parsing LaTeX
from sympy import srepr    # Import SymPy function for expression tree representation

In [2]:
lexer = Lexer(); lexer.initialize(r'\sqrt{5}(x + 2/3)^2')
print(', '.join(token for token in lexer.tokenize()))

SQRT_CMD, LEFT_BRACE, INTEGER, RIGHT_BRACE, LEFT_PAREN, SYMBOL, PLUS, RATIONAL, RIGHT_PAREN, CARET, INTEGER


In [3]:
expr = parse(r'\sqrt{5}(x + 2/3)^2', expression=True)
print(expr, ':', srepr(expr))

sqrt(5)*(x + 2/3)**2 : Mul(Pow(Integer(5), Rational(1, 2)), Pow(Add(Symbol('x'), Rational(2, 3)), Integer(2)))


#### `Grammar Derivation: (x + 2/3)^2`
```
<EXPRESSION> -> <TERM>
             -> <FACTOR>
             -> <BASE>^<EXPONENT>
             -> (<EXPRESSION>)^<EXPONENT>
             -> (<TERM> + <TERM>)^<EXPONENT>
             -> (<FACTOR> + <TERM>)^<EXPONENT>
             -> (<BASE> + <TERM>)^<EXPONENT>
             -> (<ATOM> + <TERM>)^<EXPONENT>
             -> (<VARIABLE> + <TERM>)^<EXPONENT>
             -> (<SYMBOL> + <TERM>)^<EXPONENT>
             -> (x + <TERM>)^<EXPONENT>
             -> (x + <FACTOR>)^<EXPONENT>
             -> (x + <BASE>)^<EXPONENT>
             -> (x + <ATOM>)^<EXPONENT>
             -> (x + <NUMBER>)^<EXPONENT>
             -> (x + <RATIONAL>)^<EXPONENT>
             -> (x + 2/3)^<EXPONENT>
             -> (x + 2/3)^<BASE>
             -> (x + 2/3)^<ATOM>
             -> (x + 2/3)^<NUMBER>
             -> (x + 2/3)^<INTEGER>
             -> (x + 2/3)^2
```

<a id='sandbox'></a>

# Step 2: Demonstration and Sandbox (LaTeX Parser) \[Back to [top](#toc)\]
$$\label{sandbox}$$

We implemented a wrapper function for the `parse()` method that will accept a LaTeX sentence and return a SymPy expression. Furthermore, the entire parsing module was designed for extendibility. We apply the following procedure for extending parser functionality to include an unsupported LaTeX command: append that command to the grammar dictionary in the Lexer class with the mapping regex:token, write a grammar abstraction (similar to a regular expression) for that command, add the associated nonterminal (the command name) to the command abstraction in the Parser class, and finally implement the straightforward (private) method for parsing the grammar abstraction. We shall demonstrate the extension procedure using the `\sqrt` LaTeX command.

```<SQRT> -> '\\sqrt' [ '[' <INTEGER> ']' ] '{' <EXPRESSION> '}'```
```
def _sqrt(self):
    if self.accept('LEFT_BRACKET'):
        integer = self.lexer.lexeme
        self.expect('INTEGER')
        root = Rational(1, integer)
        self.expect('RIGHT_BRACKET')
    else: root = Rational(1, 2)
    self.expect('LEFT_BRACE')
    expr = self.__expr()
    self.expect('RIGHT_BRACE')
    return Pow(expr, root)
```

In [4]:
print(parse(r'\sqrt[3]{\alpha_0}', expression=True))

alpha_0**(1/3)


In addition to expression parsing, we included support for equation parsing, which will produce a dictionary mapping LHS $\mapsto$ RHS, where LHS must be a symbol, and insert that mapping into the global namespace of the previous stack frame, as demonstrated below.

In [5]:
parse(r'x = n\sqrt{2}^n'); print(x)

2**(n/2)*n


We implemented robust error messaging using the custom `ParseError` exception, which should handle every conceivable case to identify, as detailed as possible, invalid syntax inside of a LaTeX sentence. The following are runnable examples of possible error messages (simply uncomment and run the cell):

In [6]:
# parse(r'\sqrt[*]{2}')
    # ParseError: \sqrt[*]{2}
    #                   ^
    # unexpected '*' at position 6

# parse(r'\sqrt[0.5]{2}')
    # ParseError: \sqrt[0.5]{2}
    #                   ^
    # expected token INTEGER at position 6

# parse(r'\command{}')
    # ParseError: \command{}
    #             ^
    # unsupported command '\command' at position 0

In [7]:
from warnings import filterwarnings # Import Python function for warning suppression
filterwarnings('ignore', category=OverrideWarning); del Parser.namespace['x']

In the sandbox code cell below, you can experiment with the LaTeX parser using the wrapper function `parse(sentence)`, where sentence must be a [raw string](https://docs.python.org/3/reference/lexical_analysis.html) to interpret a backslash as a literal character rather than an [escape sequence](https://en.wikipedia.org/wiki/Escape_sequence).

In [8]:
# Write Sandbox Code Here

<a id='tensor'></a>

# Step 3: Tensor Support with Einstein Notation (WIP) \[Back to [top](#toc)\]
$$\label{tensor}$$

In the following section, we demonstrate the current parser support for tensor notation using the Einstein summation convention. The first example will parse an equation for a tensor contraction, the second will parse an equation for raising an index using the metric tensor, and the third will parse an align enviroment with an equation dependency. In each example, every tensor should appear either on the LHS of an equation or inside of a configuration before appearing on the RHS of an equation. Moreover, the parser will raise an exception upon violation of the Einstein summation convention, i.e. an invalid free or bound index.

**Configuration Syntax** `% <TENSOR> [<DIMENSION>]: <SYMMETRY>, <TENSOR> [<DIMENSION>]: <SYMMETRY>, ... ;`

#### Example 1
LaTeX Source | Rendered LaTeX
:----------- | :-------------
<pre lang="latex"> h = h^\\mu{}_\\mu </pre> | $$ h = h^\mu{}_\mu $$

In [9]:
parse(r"""
    % h^\mu_\mu [4]: nosym;
    h = h^\mu{}_\mu
""")

['hUD', 'h']

In [10]:
print('h =', h)

h = hUD00 + hUD11 + hUD22 + hUD33


#### Example 2
LaTeX Source | Rendered LaTeX
:----------- | :-------------
<pre lang="latex"> v^\\mu = g^{\\mu\\nu}v_\\nu </pre> | $$ v^\mu = g^{\mu\nu}v_\nu $$

In [11]:
parse(r"""
    % g^{\mu\nu} [3]: metric, v_\nu [3];
    v^\mu = g^{\mu\nu}v_\nu
""")

['gUU', 'gDD', 'vD', 'vU']

In [12]:
print('vU =', vU)

vU = [gUU00*vD0 + gUU01*vD1 + gUU02*vD2, gUU01*vD0 + gUU11*vD1 + gUU12*vD2, gUU02*vD0 + gUU12*vD1 + gUU22*vD2]


#### Example 3
LaTeX Source | Rendered LaTeX
:----------- | :-------------
<pre lang="latex"> \\begin{align\*}<br>&emsp;&emsp;&emsp; R &= g_{ab}R^{ab} \\\\ <br>&emsp;&emsp;&emsp; G^{ab} &= R^{ab} - \\frac{1}{2}g^{ab}R <br> \\end{align\*} </pre> | $$ \begin{align*} R &= g_{ab}R^{ab} \\ G^{ab} &= R^{ab} - \frac{1}{2}g^{ab}R \end{align*} $$

In [13]:
parse(r"""
    % g_{ab} [2]: metric, R^{ab} [2]: sym01;
    \begin{align*}
        R &= g_{ab}R^{ab} \\
        G^{ab} &= R^{ab} - \frac{1}{2}g^{ab}R
    \end{align*}
""")

['gDD', 'gUU', 'RUU', 'R', 'GUU']

In [14]:
print('R =', R)

R = RUU00*gDD00 + 2*RUU01*gDD01 + RUU11*gDD11


In [15]:
display(GUU)

[[RUU00 + gDD11*(-RUU00*gDD00 - 2*RUU01*gDD01 - RUU11*gDD11)/(2*(gDD00*gDD11 - gDD01**2)),
  RUU01 - gDD01*(-RUU00*gDD00 - 2*RUU01*gDD01 - RUU11*gDD11)/(2*(gDD00*gDD11 - gDD01**2))],
 [RUU01 - gDD01*(-RUU00*gDD00 - 2*RUU01*gDD01 - RUU11*gDD11)/(2*(gDD00*gDD11 - gDD01**2)),
  RUU11 + gDD00*(-RUU00*gDD00 - 2*RUU01*gDD01 - RUU11*gDD11)/(2*(gDD00*gDD11 - gDD01**2))]]

The static variable `namespace` for the `Parser` class will provide access to the global namespace of the parser across each instance of the class.

In [16]:
Parser.namespace

OrderedDict([('hUD',
              [[hUD00, hUD01, hUD02, hUD03],
               [hUD10, hUD11, hUD12, hUD13],
               [hUD20, hUD21, hUD22, hUD23],
               [hUD30, hUD31, hUD32, hUD33]]),
             ('h', hUD00 + hUD11 + hUD22 + hUD33),
             ('gUU',
              [[gDD11/(gDD00*gDD11 - gDD01**2),
                -gDD01/(gDD00*gDD11 - gDD01**2)],
               [-gDD01/(gDD00*gDD11 - gDD01**2),
                gDD00/(gDD00*gDD11 - gDD01**2)]]),
             ('gDD', [[gDD00, gDD01], [gDD01, gDD11]]),
             ('vD', [vD0, vD1, vD2]),
             ('vU',
              [gUU00*vD0 + gUU01*vD1 + gUU02*vD2,
               gUU01*vD0 + gUU11*vD1 + gUU12*vD2,
               gUU02*vD0 + gUU12*vD1 + gUU22*vD2]),
             ('RUU', [[RUU00, RUU01], [RUU01, RUU11]]),
             ('R', RUU00*gDD00 + 2*RUU01*gDD01 + RUU11*gDD11),
             ('GUU',
              [[RUU00 + gDD11*(-RUU00*gDD00 - 2*RUU01*gDD01 - RUU11*gDD11)/(2*(gDD00*gDD11 - gDD01**2)),
                

We extended our robust error messaging using the custom `TensorError` exception, which should handle any inconsistent tensor dimension and any violation of the Einstein summation convention, specifically that a bound index must appear exactly once as a superscript and exactly once as a subscript in any single term and that a free index must appear in every term with the same position and cannot be summed over in any term. The following are runnable examples of possible error messages (simply uncomment and run the cell):

In [17]:
# parse(r"""
#     % h^{\mu\mu}_{\mu\mu} [4]: nosym;
#     h = h^{\mu\mu}_{\mu\mu}
# """)
    # TensorError: illegal bound index

# parse(r"""
#     % g^\mu_\nu [3]: sym01, v_\nu [3];
#     v^\mu = g^\mu_\nu v_\nu
# """)
    # TensorError: illegal bound index

# parse(r"""
#     % g^{\mu\nu} [3]: sym01, v_\mu [3], w_\nu [3];
#     u^\mu = g^{\mu\nu}(v_\mu + w_\nu)
# """)
    # TensorError: unbalanced free index

<a id='latex_pdf_output'></a>

# Step 4: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-LaTeX_SymPy_Conversion.pdf](Tutorial-LaTeX_SymPy_Conversion.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [18]:
import cmdline_helper as cmd    # NRPy+: Multi-platform Python command-line interface
cmd.output_Jupyter_notebook_to_LaTeXed_PDF("Tutorial-LaTeX_SymPy_Conversion")

Created Tutorial-LaTeX_SymPy_Conversion.tex, and compiled LaTeX file to PDF
    file Tutorial-LaTeX_SymPy_Conversion.pdf
